In [1]:
import numpy as np
import pandas as pd
import re
import os
import csv
from readcalc import readcalc #(https://pypi.org/project/ReadabilityCalculator/)


In [88]:
fileList = os.listdir('../Call_Transcripts_Seeking_Alpha/')
fileList

['CT_ABT_2018_q1.txt',
 'CT_ABT_2018_q2.txt',
 'CT_ABT_2018_q4.txt',
 'CT_ABT_2019_q1.txt',
 'CT_AXP_2018_q1.txt',
 'CT_AXP_2018_q2.txt',
 'CT_AXP_2018_q3.txt',
 'CT_AXP_2018_q4.txt',
 'CT_AXP_2019_q1.txt',
 'CT_BA_2018_q1.txt',
 'CT_BA_2018_q2.txt',
 'CT_BA_2018_q3.txt',
 'CT_BA_2018_q4.txt',
 'CT_BA_2019_q1.txt',
 'CT_COST_2018_q1.txt',
 'CT_COST_2018_q2.txt',
 'CT_COST_2018_q3.txt',
 'CT_COST_2018_q4.txt',
 'CT_DIS_2018_q1.txt',
 'CT_DIS_2018_q2.txt',
 'CT_DIS_2018_q3.txt',
 'CT_DIS_2018_q4.txt',
 'CT_DIS_2019_q1.txt',
 'CT_ETN_2018_q1.txt',
 'CT_ETN_2018_q2.txt',
 'CT_ETN_2018_q3.txt',
 'CT_ETN_2018_q4.txt',
 'CT_ETN_2019_q1.txt',
 'CT_FB_2018_q1.txt',
 'CT_FB_2018_q2.txt',
 'CT_FB_2018_q3.txt',
 'CT_FB_2018_q4.txt',
 'CT_FB_2019_q1.txt',
 'CT_FDX_2018_q1.txt',
 'CT_FDX_2018_q2.txt',
 'CT_FDX_2018_q3.txt',
 'CT_FDX_2018_q4.txt',
 'CT_FDX_2019_q1.txt',
 'CT_GE_2018_q1.txt',
 'CT_GE_2018_q2.txt',
 'CT_GE_2018_q3.txt',
 'CT_GE_2018_q4.txt',
 'CT_GE_2019_q1.txt',
 'CT_GOOGL_2018_q3.txt

In [89]:
def extractInfoFromCallTscript(filename):
    filepath = '../Call_Transcripts_Seeking_Alpha/' + filename 
    file = open(filepath, 'r', encoding='cp1252')

    fullCallTranscript = file.readlines()   #.split('Question-and-Answer Session')
    print('Length of Call Transcript:' + str(len(fullCallTranscript)))
    
    #Extract Q & A Session
    
    try:
        q_and_a_index = fullCallTranscript.index("Question-and-Answer Session\n")
    except ValueError: 
        q_and_a_index = 'None'
    if q_and_a_index== 'None':
        try:
            q_and_a_index = fullCallTranscript.index("Question-and-Answer Session \n")
        except ValueError: 
            print('Transcript format is different than standard, modify code')
        
    # Divide the transcript into initial summary and Q & A Session
    q_and_a = fullCallTranscript[q_and_a_index:]
    summary = fullCallTranscript[:q_and_a_index]
    
#     print(summary)
    
    # Extract Lists of executives and analysts on call
    # Note: SeekingAlpha changed the format from Executive --> Company Participants and Analysts --> Conference Call Participants
    # between 2018 and 2019. The try-except blocks handle that change
    try:
        execIndex = (summary.index("Executives\n"))
    except ValueError: 
        execIndex = 'None'
    if execIndex == 'None':
        try:
            execIndex = summary.index("Company Participants\n")
        except ValueError: 
            execIndex = 'None'
    if execIndex == 'None':
        try:                 
            execIndex = summary.index("Company Representatives\n")
        except ValueError: 
            print('Transcript format is different than standard, modify code')

     
    #################################################################
    try:
        analystIndex = (summary.index("Analysts\n"))
    except ValueError: 
        analystIndex = 'None'
    if analystIndex == 'None':
        try:
            analystIndex = summary.index("Conference Call Participants\n")
        except ValueError: 
            print('Transcript format is different than standard, modify code')
            
    #################################################################

    operatorIndex = summary.index("Operator\n")
    
    
    # CREATE DICTIONARIES FOR EXECUTIVES AND ANALYSTS
    executives = summary[execIndex + 1 : analystIndex]
    print(executives)
    executiveDict = {}
    for e in executives:
#         print(e)
#         e = str.replace(e, '–', '-')
        try:
            name, position = e.split('-')
        except:
            name, position = e.split('–')
             
        executiveDict[name.strip()] = position.strip() + '|'
    print("WTF")
    print(executiveDict)

    analysts = summary[analystIndex +1: operatorIndex]
    print(analysts)

    analystsDict = {}
    for a in analysts:
        try:
            name, company = a.split('-')
        except:
            name, company = a.split('–')
#         name, company = a.split('-')
        analystsDict[name.strip()] = company.strip() + '|'
    analystsDict

    # Store Executives and their TalkTracks in a dictionary
    # elem,next_elem in zip(li, li[1:]+[li[0]]):
    for item,next_item in zip(q_and_a, q_and_a[1:]+[q_and_a[0]]):
    #     print(item)
        if item.split('-')[0].strip() in executiveDict.keys():

            val = executiveDict[item.split('-')[0].strip()]
            executiveDict[item.split('-')[0].strip()] = val + ' ' + next_item
    #         print(item, next_item)

    print('Number of Executives on call' + str(len(executiveDict.keys())))
    
    # Get and store the answers of the analysts in the analystsDictionary

    # elem,next_elem in zip(li, li[1:]+[li[0]]):
    for item,next_item in zip(q_and_a, q_and_a[1:]+[q_and_a[0]]):
    #     print(item)
        if item.split('-')[0].strip() in analystsDict.keys():
    #         print(item.strip())
            val = analystsDict[item.split('-')[0].strip()]
            analystsDict[item.split('-')[0].strip()] = val + ' ' + next_item
    #         print(item, next_item)

    print('Number of Analysts on call' + str(len(analystsDict.keys())))
    
    
    # CONVERT DICTIONARIES TO A SINGLE UNIFIED DATAFRAME
    
    # For Analysts
    analysts_df = pd.DataFrame(list(analystsDict.items()), columns=['Speaker', 'TalkTrack'])

    split_values = analysts_df["TalkTrack"].str.split("|", n = 1, expand = True) 
    # making seperate first name column from new data frame 
    analysts_df["Org_Title"]= split_values[0] 

    # making seperate last name column from new data frame 
    analysts_df["TalkTrack"]= split_values[1] 


    analysts_df["SpeakerType"] = "Analyst"
    analysts_df
    
    # For Executives
    executives_df = pd.DataFrame(list(executiveDict.items()), columns=['Speaker', 'TalkTrack'])


    split_values = executives_df["TalkTrack"].str.split("|", n = 1, expand = True) 
    # making seperate first name column from new data frame 
    executives_df["Org_Title"]= split_values[0] 

    # making seperate last name column from new data frame 
    executives_df["TalkTrack"]= split_values[1] 

    executives_df["SpeakerType"] = "Executive"
    executives_df
    
    # COMBINE BOTH INTO ONE DATAFRAME
    complete_df = analysts_df.append(executives_df, ignore_index = True)
    complete_df["CallTranscriptID"] = str.replace(filename, '.txt', '')
    print(str.replace(filename, '.txt', ''))
    
    # Split and save Ticker, Year and Quarter as well
    # new data frame with split value columns 
    new = complete_df["CallTranscriptID"].str.split("_", n = 4, expand = True) 
#     print("New" + new)
  
    # making separate first name column from new data frame 
    complete_df["Ticker"]= new[1] 

    # making separate last name column from new data frame 
    complete_df["Year"]= new[2]
    
    # making separate last name column from new data frame 
#     qtr = (new[3])
#     print(str.replace(qtr, '.txt', ''))
    complete_df["Quarter"]= new[3]

    # Dropping old Name columns 
#     t.drop(columns =["test"], inplace = True) 
    
    #Return dataframe
    return complete_df
    # Write output dataframe



In [90]:
import pandas as pd
cNames = ["Speaker", "TalkTrack", "Org_Title", "SpeakerType", "CallTranscriptID", "Ticker", "Year", "Quarter"]

# Creating an empty Dataframe with column names only
masterDF = pd.DataFrame(columns=cNames)

for file in fileList:
    outputDF = extractInfoFromCallTscript(file)
    masterDF = masterDF.append(outputDF, ignore_index=True)
#     print(outputDF)
    


Length of Call Transcript:168
['Scott Leinenweber - Vice President, Investor Relations\n', 'Miles White - Chairman, Chief Executive Officer\n', 'Brian Yoor - Executive Vice President, Finance, Chief Financial Officer\n']
{'Scott Leinenweber': 'Vice President, Investor Relations|', 'Miles White': 'Chairman, Chief Executive Officer|', 'Brian Yoor': 'Executive Vice President, Finance, Chief Financial Officer|'}
['Mike Weinstein - JPMorgan\n', 'David Lewis - Morgan Stanley\n', 'Lawrence Biegelsen - Wells Fargo\n', 'David Lewis - Morgan Stanley\n', 'Rick Wise - Stifel \n', 'Chris Pasquale - Guggenheim\n', 'Glenn Novarro - RBC Capital Markets\n']
Number of Executives on call3
Number of Analysts on call6
CT_ABT_2018_q1
Length of Call Transcript:209
['Scott Leinenweber - Vice President, Investor Relations\n', 'Miles White - Chairman, Chief Executive Officer\n', 'Brian Yoor - Executive Vice President, Finance, Chief Financial Officer\n']
{'Scott Leinenweber': 'Vice President, Investor Relations

Length of Call Transcript:212
['Lowell Singer - The Walt Disney Co.\n', 'Robert A. Iger - The Walt Disney Co.\n', 'Christine M. McCarthy - The Walt Disney Co.\n']
{'Lowell Singer': 'The Walt Disney Co.|', 'Robert A. Iger': 'The Walt Disney Co.|', 'Christine M. McCarthy': 'The Walt Disney Co.|'}
['Benjamin Daniel Swinburne - Morgan Stanley & Co. LLC\n', 'Jessica Jean Reif Cohen - Bank of America Merrill Lynch\n', 'Michael B. Nathanson - MoffettNathanson LLC\n', 'Alexia S. Quadrani - JPMorgan Securities LLC\n', 'Marci L. Ryvicker - Wells Fargo Securities LLC\n', 'Todd Michael Juenger - Sanford C. Bernstein & Co. LLC\n', 'Steven Cahall - RBC Capital Markets LLC\n', 'Kannan Venkateshwar - Barclays Capital, Inc.\n']
Number of Executives on call3
Number of Analysts on call8
CT_DIS_2018_q2
Length of Call Transcript:246
['Lowell Singer - The Walt Disney Co.\n', 'Robert A. Iger - The Walt Disney Co.\n', 'Christine M. McCarthy - The Walt Disney Co.\n']
{'Lowell Singer': 'The Walt Disney Co.|', '

ValueError: not enough values to unpack (expected 2, got 1)

In [91]:
masterDF.columns

Index(['Speaker', 'TalkTrack', 'Org_Title', 'SpeakerType', 'CallTranscriptID',
       'Ticker', 'Year', 'Quarter'],
      dtype='object')

In [93]:

# import sys


# from unidecode import unidecode
# clean = unidecode(u'Some text: \u2018\u2019\u2013\u03a9')


# s="Mark Zuckerberg – Chief Executive Officer"
# # str.replace(s, '–', '-')
# clean1 = unidecode(s)
# # s.decode("utf8").replace(u"\u2212", "-")
# print(s)
# n, p = s.split('– ')
# print(n)


In [94]:
# masterDF =masterDF.drop('TalkTrack', axis=1)

In [11]:
masterDF.to_excel('outputs/Analyst_Exec_Breakup.xlsx', index=False )

# Sentiment Analysis

In [75]:
%run SentimentAnalysis_CallTranscripts.ipynb

The results with negation check:

The # of positive words: 44
The # of negative words: 17
The list of found positive words: ['positive', 'good', 'good', 'strong', 'stronger', 'great', 'great', 'impressed', 'efficiency', 'opportunities', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'gaining', 'positive', 'strong', 'stronger', 'great', 'great', 'impressed', 'efficiency', 'opportunities', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'gaining', 'positive', 'good', 'good', 'strong', 'stronger', 'great', 'great', 'impressed', 'efficiency', 'opportunities']
The list of found negative words: ['ill', 'question', 'question', 'great (with negation)', 'confusion', 'firing', 'ill', 'question', 'question', 'great (with negation)', 'confusion', 'firing', 'ill', 'question', 'question', 'great (with negation)', 'confusion']

The list of positive words: ['positive', 'good', 'strong', 'stronger', 'great', 'impressed', 'efficiency', 'opportunities', 'gaining']
The # of positive words

In [113]:
x['tone1'] = x['TalkTrack'].apply(tone_count_with_negation_check)


In [119]:
x = pd.concat([x.drop(['tone1'],axis=1),
           x['tone1'].apply(pd.Series).rename(columns={0:'total_words',1:'pos_word_count',2:'neg_word_count', 
                                                       3:'pos_words', 4:'neg_words'})],axis=1)

In [120]:
x.head()

,Speaker,TalkTrack,Org_Title,SpeakerType,CallTranscriptID,Ticker,Year,Quarter,GunningFogIndex,total_words,pos_word_count,neg_word_count,pos_words,neg_words,total_words,pos_word_count,neg_word_count,pos_words,neg_words
0,Mike Weinstein,Let me start if I can on couple of product la...,JPMorgan,Analyst,CT_ABT_2018_q1,ABT,2018,q1,8.299465,166,2,2,"[rebound, good]","[weak, question]",166,2,2,"[rebound, good]","[weak, question]"
1,David Lewis,"Hello.\n So, Miles, I wanted to start with di...",Morgan Stanley,Analyst,CT_ABT_2018_q1,ABT,2018,q1,7.829814,164,0,0,[],[],164,0,0,[],[]
2,Lawrence Biegelsen,"Good morning, guys. Thanks for taking the que...",Wells Fargo,Analyst,CT_ABT_2018_q1,ABT,2018,q1,8.985041,247,6,5,"[good, improved, improved, good, spectacular, ...","[question, questions, challenges, question, di...",247,6,5,"[good, improved, improved, good, spectacular, ...","[question, questions, challenges, question, di..."
3,Rick Wise,"Miles, just a big question to start off with....",Stifel,Analyst,CT_ABT_2018_q1,ABT,2018,q1,11.373785,175,1,1,[opportunity],[question],175,1,1,[opportunity],[question]
4,Chris Pasquale,Thanks. One question on Libre and then one on...,Guggenheim,Analyst,CT_ABT_2018_q1,ABT,2018,q1,9.831278,132,2,2,"[strong, opportunity]","[question, breakdown]",132,2,2,"[strong, opportunity]","[question, breakdown]"


In [78]:
# import pandas as pd
# x = pd.read_excel('output/Analyst_Exec_Breakup.xlsx')

In [114]:
# y = tone_count_with_negation_check(x['TalkTrack'][0])

# Test the Code

In [82]:
# tone_count_with_negation_check(lmdict, text)

# Gunning Fog Index Computation

In [121]:
from readcalc import readcalc #(https://pypi.org/project/ReadabilityCalculator/)
# calc = readcalc.ReadCalc("This is a simple text.")


TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 22


In [122]:
def get_GF_index(text):
    calc_1 = readcalc.ReadCalc(text)
    gf = calc_1.get_gunning_fog_index()
    return gf

In [123]:
# print(calc_1.get_gunning_fog_index)
x['GunningFogIndex'] = x['TalkTrack'].apply(get_GF_index)

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 923
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 857
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1395
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 980
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 715
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 566
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1481
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 10191
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 91
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 962
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 934
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1019
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text:

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 550
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 661
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 954
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 892
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1229
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1165
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 651
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 438
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 735
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 525
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1357
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 731
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 

TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 3902
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 680
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 796
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 774
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 864
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 967
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 954
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 1405
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 697
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 406
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 544
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 739
TEXT IS NOT BEING PRE PROCESSED
type(text) == <class 'str'> ; Size Text: 5

In [124]:
x.head()

,Speaker,TalkTrack,Org_Title,SpeakerType,CallTranscriptID,Ticker,Year,Quarter,GunningFogIndex,total_words,pos_word_count,neg_word_count,pos_words,neg_words,total_words,pos_word_count,neg_word_count,pos_words,neg_words
0,Mike Weinstein,Let me start if I can on couple of product la...,JPMorgan,Analyst,CT_ABT_2018_q1,ABT,2018,q1,8.299465,166,2,2,"[rebound, good]","[weak, question]",166,2,2,"[rebound, good]","[weak, question]"
1,David Lewis,"Hello.\n So, Miles, I wanted to start with di...",Morgan Stanley,Analyst,CT_ABT_2018_q1,ABT,2018,q1,7.829814,164,0,0,[],[],164,0,0,[],[]
2,Lawrence Biegelsen,"Good morning, guys. Thanks for taking the que...",Wells Fargo,Analyst,CT_ABT_2018_q1,ABT,2018,q1,8.985041,247,6,5,"[good, improved, improved, good, spectacular, ...","[question, questions, challenges, question, di...",247,6,5,"[good, improved, improved, good, spectacular, ...","[question, questions, challenges, question, di..."
3,Rick Wise,"Miles, just a big question to start off with....",Stifel,Analyst,CT_ABT_2018_q1,ABT,2018,q1,11.373785,175,1,1,[opportunity],[question],175,1,1,[opportunity],[question]
4,Chris Pasquale,Thanks. One question on Libre and then one on...,Guggenheim,Analyst,CT_ABT_2018_q1,ABT,2018,q1,9.831278,132,2,2,"[strong, opportunity]","[question, breakdown]",132,2,2,"[strong, opportunity]","[question, breakdown]"


In [126]:
x.to_excel('outputs/CallTranscripts_DataFrame_Analyst_Exec_Breakup.xlsx', index=False )